#Generate_label.py

In [1]:
import pandas as pd
import os

audio_dir = 'C:/Users/raada/Documents/Pray&Hope/data/audio'
# print(f'audio_dir: {audio_dir}')

emotion_map = {
    '01': 'neutral',
    '02': 'calm',
    '03': 'happy',
    '04': 'sad',
    '05': 'angry',
    '06': 'fearful',
    '07': 'disgust',
    '08': 'surprised',
}

data = []

for root, dirs, files in os.walk(audio_dir):
    for file in files:
        if file.endswith('.wav'):
            parts = file.split('-')
            emotion_code = parts [2]
            emotion = emotion_map.get(emotion_code, 'unknown')
            #Store the  relative path from data/audio
            relative_path = os.path.relpath(os.path.join(root, file), audio_dir)
            data.append({'filename': relative_path, 'emotion': emotion})

df = pd.DataFrame(data)


df.to_csv(os.path.join(audio_dir, 'labels.csv'), index=False)

print("file banaise")



file banaise


#preprocessor_audio

In [2]:
import os
import pandas as pd
import librosa
import numpy as np

# path to the audio directory and labels file
audio_dir = 'C:/Users/raada/Documents/Pray&Hope/data/audio'
labels_df = pd.read_csv(os.path.join(audio_dir, 'labels.csv'))

#initialize lists to hold features and labels
features = []
labels = []

# loop through each row in the labels DataFrame
for index, row in labels_df.iterrows():
    audio_file = os.path.join(audio_dir, row['filename'])
    emotion = row['emotion']

    #load the audio file and standardize it
    y, sr = librosa.load(audio_file, sr=16000) #resample to 16KHz
    y = librosa.util.normalize(y)  # normalize the audio signal

    # extract MFCCs
    mfccs = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=13)
    mfccs_mean = np.mean(mfccs, axis=1)

    features.append(mfccs_mean)
    labels.append(emotion)

#conver to numpy arrays
X = np.array(features)
y =np.array(labels)

np.save(os.path.join(audio_dir, 'x.npy'), X)
np.save(os.path.join(audio_dir, 'y.npy'), y)

#train_audio_model.py

In [3]:
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report
import joblib

audio_dir = 'C:/Users/raada/Documents/Pray&Hope/data/audio'
X = np.load(os.path.join(audio_dir, 'x.npy'))
y = np.load(os.path.join(audio_dir, 'y.npy'))

# split data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

#train a model( Random Forest Classifier)
model = RandomForestClassifier()
model.fit(X_train, y_train)

#evaluate the model
y_pred =model.predict(X_test)
print(classification_report(y_test, y_pred))

dump_dir = 'C:/Users/raada/Documents/Pray&Hope/models'
joblib.dump(model, os.path.join(dump_dir, 'audio_emotion_model.pkl'))
print("Model trained and saved successfully.")


              precision    recall  f1-score   support

       angry       0.74      0.60      0.66        42
        calm       0.59      0.75      0.66        44
     disgust       0.38      0.50      0.43        32
     fearful       0.47      0.62      0.53        32
       happy       0.55      0.53      0.54        34
     neutral       0.45      0.25      0.32        20
         sad       0.50      0.41      0.45        39
   surprised       0.43      0.36      0.39        45

    accuracy                           0.52       288
   macro avg       0.51      0.50      0.50       288
weighted avg       0.52      0.52      0.51       288

Model trained and saved successfully.


#Predict_audio

In [4]:
import os
import librosa
import numpy as np
import joblib


model = joblib.load('C:/Users/raada/Documents/Pray&Hope/models/audio_emotion_model.pkl')
#path to the new audio file
audio_file =  'C:/Users/raada/Documents/Pray&Hope/data/predict_test/03-01-06-01-02-01-01.wav'

y, sr =librosa.load(audio_file, sr=16000)  # load and resample the audio file
mfccs = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=13)
mfccs_mean = np.mean(mfccs, axis=1)

X_new = mfccs_mean.reshape(1, -1)  # reshape for prediction
emotion = model.predict(X_new)

print(f'Predicted emotion: {emotion[0]}')

Predicted emotion: fearful
